## Exemplo Básico

In [ ]:
import numpy as np
import pandas as pd
series = pd.Series(np.arange(50))

series

In [ ]:
from src.tde import UnivariateTDE

# numeros de lags
n_steps = 5
# horizonte
n_output = 3
# no de variables -- 1 porque a série é univariada
n_features = 1

data = UnivariateTDE(series, k=n_steps, horizon=n_output)

data.head()

In [ ]:
is_future = data.columns.str.contains('\+')
X = data.iloc[:,~is_future].values
Y = data.iloc[:,is_future].values

X[:3]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# building a custom LSTM network
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
# final layer is contains as many units as the size of the output
model.add(Dense(n_output))
# compiling with ADAM and optimizing to MSE
model.compile(optimizer='adam', loss='mse')

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], n_features))
X[:3]

In [ ]:
Y[:3]

In [ ]:
model.fit(X, Y, epochs=10, verbose=1)

In [ ]:
# lstm but with 2 blocks

model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(20, activation='relu'))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

model.fit(X, Y, epochs=10, verbose=1)

In [ ]:
model.summary()

## Using State-of-the-art Methods

In [ ]:
# métodos avançados

from gluonts.model.n_beats import NBEATSEnsembleEstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import make_evaluation_predictions
from gluonts.evaluation import Evaluator
from gluonts.dataset.repository.datasets import get_dataset, dataset_names

In [ ]:
from pprint import pprint
pprint(dataset_names)

In [ ]:
dataset = get_dataset("tourism_quarterly", regenerate=True)
horizon = 4

In [ ]:
# versão MUITO simplificada
estimator = NBEATSEnsembleEstimator(freq='Q',
                                    prediction_length=horizon,
                                    meta_context_length=[2 * horizon],
                                    meta_loss_function=['MASE'],
                                    meta_bagging_size=2,
                                    num_stacks=30,
                                    num_blocks=[1],
                                    trainer=Trainer(
                                        learning_rate=0.001,
                                        # epochs=30,
                                        epochs=2, # to get some quick output
                                        batch_size=1024
                                    ),
                                    widths=[64],
                                    sharing=[False],
                                    expansion_coefficient_lengths=[32],
                                    stack_types=['G'])

In [ ]:
model = estimator.train(dataset.train)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,  
    predictor=model,  # modelo
    num_samples=1,  # number of sample paths we want for evaluation
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.test))

In [ ]:
agg_metrics